In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

## 영문텍스트에 anki단어장 만들기

In [24]:
# 함수 리팩토링
default_text = """
It's always difficult to keep personal prejudice out of a thing like this. And wherever you run into it, prejudice always obscures the truth. I don't really know what the truth is. I don't suppose anybody will ever really know. Nine of us now seem to feel that the defendant is innocent, but we're just gambling on probabilities - we may be wrong. We may be trying to let a guilty man go free, I don't know. Nobody really can. But we have a reasonable doubt, and that's something that's very valuable in our system. No jury can declare a man guilty unless it's sure.
"""

def correct_news_title(model='gpt-4o-mini', temperature=1, top_p=1, user_input_text=default_text):
    # 역할/페르소나/지시사항
    system_message = """
    영어 텍스트를 입력하면, 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성해라.

    ==지시사항==
    - LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.
    - 너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.
    - 각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.
    - 각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.
    - 출력 형식은 다음과 같다. input=The committee will convene next week to discuss the new policy proposals and reach a consensus.
                              output=convene;to come together for a meeting;verb;The committee will convene next week.\n
                                     consensus;general agreement;noun;The group reached a consensus after a long discussion.\n
                                     proposal;a suggestion or plan;noun;He submitted a proposal for the new project.
    """

    user_message = f'{user_input_text}'

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0

    )
    return response.choices[0].message.content



output = correct_news_title()
print(output)

prejudice;preconceived opinion that is not based on reason or actual experience;noun;It's always difficult to keep personal prejudice out of a thing like this.

obscure;to make something difficult to see or understand;verb;Prejudice always obscures the truth.

defendant;a person accused in a court of law;noun;Nine of us now seem to feel that the defendant is innocent.

probabilities;the likelihood or chance of something happening;noun;We're just gambling on probabilities - we may be wrong.

reasonable doubt;the standard of evidence required to validate a criminal conviction;phrase;But we have a reasonable doubt, and that's something that's very valuable in our system.

guilty;responsible for a specified wrongdoing;adjective;We may be trying to let a guilty man go free.


In [25]:
import pandas as pd

lines = output.strip().split('\n')
rows = [line.split(';') for line in lines]

df = pd.DataFrame(rows, columns=['단어', '의미', '품사', '예시'])
df

,단어,의미,품사,예시
0,prejudice,preconceived opinion that is not based on reas...,noun,It's always difficult to keep personal prejudi...
1,,None,None,None
2,obscure,to make something difficult to see or understand,verb,Prejudice always obscures the truth.
3,,None,None,None
4,defendant,a person accused in a court of law,noun,Nine of us now seem to feel that the defendant...
5,,None,None,None
6,probabilities,the likelihood or chance of something happening,noun,We're just gambling on probabilities - we may ...
7,,None,None,None
8,reasonable doubt,the standard of evidence required to validate ...,phrase,"But we have a reasonable doubt, and that's som..."
9,,None,None,None


'r'